In [1]:
%matplotlib inline

In [2]:
from eam_core.util import h5load
import os

[root                ][INFO   ]  Configured logging from /home/james/eam-core-provenance/src/eam_core/logconf.yml (log_configuration.py:30)


In [43]:
# should point to a /result_data_energy.hdf5 output file
h5path = ''
d,m = h5load(h5path)
# d,m = h5load('../../../output/1/CI_model/20200508-114814/result_data_energy.hdf5')
# os.getcwd()

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 411, in H5Fcreate
    invalid file name

End of HDF5 error back trace

Unable to open/create file ''

In [6]:
d.columns

MultiIndex(levels=[['CDN', 'Internet Network', 'Laptop'], ['gigawatt_hour']],
           codes=[[0, 1, 2], [0, 0, 0]],
           names=[None, 'unit'])

In [8]:
lci = d['CDN'].groupby(level=['time']).quantile(.05)

In [14]:
lci=d['CDN'].groupby(level=['time']).quantile(.05)
uci=d['CDN'].groupby(level=['time']).quantile(.95)

In [12]:
_mean = d['CDN'].mean(level='time')

In [13]:
d['CDN'].index.unique(level='time')

DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01'],
              dtype='datetime64[ns]', name='time', freq=None)

In [15]:
lci['gigawatt_hour']

time
2019-01-01    0.001073
2019-02-01    0.001092
2019-03-01    0.001112
2019-04-01    0.001133
2019-05-01    0.001154
2019-06-01    0.001175
2019-07-01    0.001197
2019-08-01    0.001219
2019-09-01    0.001241
2019-10-01    0.001264
2019-11-01    0.001287
2019-12-01    0.001311
2020-01-01    0.001335
Name: gigawatt_hour, dtype: float64

In [18]:
ax = _mean.plot()
ax.fill_between(d['CDN'].index.unique(level='time'), lci['gigawatt_hour'], uci['gigawatt_hour'], alpha=.25)

In [23]:
d

,,CDN,Internet Network,Laptop
,unit,gigawatt_hour,gigawatt_hour,gigawatt_hour
time,samples,,,
2019-01-01,0,0.001073,0.001073,1.681290e-08
2019-02-01,0,0.001092,0.001092,1.680046e-08
2019-03-01,0,0.001112,0.001112,1.678922e-08
2019-04-01,0,0.001133,0.001133,1.677679e-08
2019-05-01,0,0.001154,0.001154,1.676476e-08
2019-06-01,0,0.001175,0.001175,1.675233e-08
2019-07-01,0,0.001197,0.001197,1.674031e-08
2019-08-01,0,0.001219,0.001219,1.672789e-08


In [21]:
d.pint.quantify(level=-1)

,,CDN,Internet Network,Laptop
time,samples,,,
2019-01-01,0,0.0010725044762169328,0.0010725044762169328,1.681290433461843e-08
2019-02-01,0,0.0010922301350473553,0.0010922301350473553,1.6800459327329347e-08
2019-03-01,0,0.0011123583146865222,0.0011123583146865222,1.6789222635947523e-08
2019-04-01,0,0.0011328166753811993,0.0011328166753811993,1.677678639803399e-08
2019-05-01,0,0.0011536649410646168,0.0011536649410646168,1.6764755716170496e-08
2019-06-01,0,0.0011748826906932117,0.0011748826906932117,1.675232854489948e-08
2019-07-01,0,0.0011965048226673092,0.0011965048226673092,1.674030663720617e-08
2019-08-01,0,0.0012185101366089903,0.0012185101366089903,1.672788853257767e-08
2019-09-01,0,0.0012409199866173781,0.0012409199866173781,1.671547503558717e-08


In [24]:
import json
from eam_core.SimulationRunner import SimulationRunner
from eam_core.YamlLoader import YamlLoader
from eam_core.common_graphical_analysis import load_metadata
from eam_core.core import complex_decoder, complex_encoder, ServiceModel, ScalarDataSource, ExcelDataSource, \
    SimulationControl

import numpy as np
from eam_core.util import create_output_folder, load_as_df_qantity

In [44]:
# should point to the output /default folder
# eg eam-core-provenance/output/CI_model/20210712-105542/default
model_output_directory = ''


In [37]:
base_dir = '/'
metadata = load_metadata(model_output_directory, base_dir=base_dir)
load_data = load_as_df_qantity
variable = 'energy'
q_data = load_data(f'{model_output_directory}/result_data_{variable}.hdf5')
data = q_data[0]
unit = str(q_data[1])

# monthly mean values for all processes
mean = data.mean(level='time').mean()


In [38]:
q_data[1]

{'CDN': <Unit('gigawatt_hour')>,
 'Internet Network': <Unit('gigawatt_hour')>,
 'Laptop': <Unit('gigawatt_hour')>}

In [39]:
_d = data.mean(level='time').sum()

/home/james/envs/eam-env-2/lib/python3.8/site-packages/numpy/core/_asarray.py:85: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order)


In [40]:
{k:v for k,v in _d.items()}

{'CDN': 0.015590618413574961 <Unit('gigawatt_hour')>,
 'Internet Network': 0.015590618413574961 <Unit('gigawatt_hour')>,
 'Laptop': 2.1762038415119822e-07 <Unit('gigawatt_hour')>}

In [41]:
list(_d.keys())

['CDN', 'Internet Network', 'Laptop']

In [42]:
list(_d.values)

[0.015590618413574961 <Unit('gigawatt_hour')>,
 0.015590618413574961 <Unit('gigawatt_hour')>,
 2.1762038415119822e-07 <Unit('gigawatt_hour')>]